In [ ]:
import nltk
import numpy as np
import pandas as pd

In [ ]:
football= pd.read_csv("/content/drive/MyDrive/footballsowmya.csv")
cricket = pd.read_csv("/content/drive/MyDrive/cricketsowmya.csv")
tennis = pd.read_csv("/content/drive/MyDrive/tennisssowmya.csv")

In [ ]:
football = football['title']
cricket = cricket['title']
tennis = tennis['title']

In [ ]:
football = football.to_frame()
football['domain'] = 'football'

In [ ]:
tennis = tennis.to_frame()
tennis['domain'] = 'tennis'

In [ ]:
cricket = cricket.to_frame()
cricket['domain'] = 'cricket'

In [ ]:
concatenated = pd.concat([football, cricket])

In [ ]:
final = pd.concat([concatenated, tennis])

In [ ]:
final.info()

In [ ]:
final.to_csv('/content/drive/MyDrive/IR_dataset_for_preprossing.csv')

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/IR_dataset_for_preprossing.csv")


In [ ]:
data.head()

In [ ]:
pip install emot

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import re     
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
# nlp = en_core_web_md.load()

In [ ]:
pip install contractions

In [ ]:
pip install word2number

In [ ]:
pip install unidecode


In [ ]:
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import gensim.downloader as api
import en_core_web_md
import contractions
import pandas as pd
from itertools import count

In [ ]:
nlp = en_core_web_md.load()

In [ ]:
def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())



def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
#     text = list(cont.expand_texts([text], precise=True))[0]
    return text


def text_preprocessing(text, contractions=False, 
                       convert_num=False, extra_whitespace=False, 
                       lemmatization=False, lowercase=False, punctuations=False,
                       remove_num=False, special_chars=True, 
                       stop_words=False):
    """preprocess text with default option set to true for all steps"""
    
    text = str(text)
    
    if extra_whitespace == True: #remove extra whitespaces
        text = remove_whitespace(text)
    if contractions == True: #expand contractions
        text = expand_contractions(text)
    if lowercase == True: #convert all characters to lowercase
        text = text.lower()
    

    doc = nlp(text) #tokenise text

    clean_text = []
    ct = ''
    
    for token in doc:
        flag = True
        edit = token.text
        # remove stop words
        if stop_words == True and token.is_stop and token.pos_ != 'NUM': 
            flag = False
        # remove punctuations
        if punctuations == True and token.pos_ == 'PUNCT' and flag == True: 
            flag = False
        # remove special characters
        if special_chars == True and token.pos_ == 'SYM' and flag == True: 
            flag = False
        # remove numbers
        if remove_num == True and (token.pos_ == 'NUM' or token.text.isnumeric()) \
        and flag == True:
            flag = False

        # convert number words to numeric numbers
        if convert_num == True and token.pos_ == 'NUM' and flag == True:
            edit = w2n.word_to_num(token.text)
        # convert tokens to base form
        elif lemmatization == True and token.lemma_ != "-PRON-" and flag == True:
            edit = token.lemma_
        # append tokens edited and not removed to list 
        if edit != "" and flag == True:
            clean_text.append(edit)    
     
    for i in clean_text:
      ct += i+' '
    ct = remove_whitespace(ct)
    return ct

In [ ]:
def preprocess_and_save(ds):
  #ds = ds.astype(str)
  ds['title'] = ds['title'].map(text_preprocessing)
  
  ds.to_csv("/content/drive/MyDrive/IR_domain_custom_dataset_processed.csv")
  print("save successful")

In [ ]:
preprocess_and_save(data)

In [ ]:
data.head()

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/IR_domain_custom_dataset_processed.csv")

In [ ]:
data.head()

In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string

In [ ]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(data['title'], data['domain'])

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(data['title'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)


In [ ]:
from sklearn import *
from sklearn.naive_bayes import MultinomialNB

mod = MultinomialNB()
clf = mod.fit(xtrain_tfidf, train_y)

In [ ]:
from sklearn.metrics import accuracy_score

predicted = mod.predict(xvalid_tfidf)
print("Accuracy:", accuracy_score(predicted, valid_y))

In [ ]:
import pickle
# save the classifier
with open('/content/drive/MyDrive/NB_classifier.pkl', 'wb') as fid:
    pickle.dump(mod, fid)  


In [ ]:
import pickle
# load it again
with open('/content/drive/MyDrive/NB_classifier.pkl', 'rb') as fid:
    Nb_loaded = pickle.load(fid)

In [ ]:
Nb_loaded

In [ ]:
query = ['virat kohli']
query_transform = tfidf_vect.transform(query)

y_pred = Nb_loaded.predict(query_transform)
print(y_pred)